## 1. data loader

In [10]:
import dgl
import torch
from dgl.data import AsNodePredDataset

In [3]:
def load_cora():
    data0 = dgl.data.CSVDataset('../graph_dgl/cora_csv/')
    data = AsNodePredDataset(data0, split_ratio=(0.5,0.2,0.3))
    g = data[0]
    g.ndata["features"] = g.ndata.pop("feat")
    g.ndata["labels"] = g.ndata.pop("label")
    return g, data.num_classes

In [4]:
def load_reddit(self_loop=True, raw_dir='~/.dgl/'):
    from dgl.data import RedditDataset

    # load reddit data
    data = RedditDataset(self_loop=self_loop, raw_dir=raw_dir)
    g = data[0]
    g.ndata["features"] = g.ndata.pop("feat")
    g.ndata["labels"] = g.ndata.pop("label")
    return g, data.num_classes

In [8]:
g, n_classes = load_cora()

Done loading data from cached files.


In [11]:
n_edges = g.num_edges()
reverse_eids = torch.cat([
    torch.arange(n_edges // 2, n_edges),
    torch.arange(0, n_edges // 2)])

In [13]:
n_edges

5429

In [12]:
reverse_eids

tensor([2714, 2715, 2716,  ..., 2711, 2712, 2713])

In [14]:
train_nid = th.nonzero(g.ndata['train_mask'], as_tuple=True)[0]
val_nid = th.nonzero(g.ndata['val_mask'], as_tuple=True)[0]
test_nid = th.nonzero(~(g.ndata['train_mask'] | g.ndata['val_mask']), as_tuple=True)[0]

In [15]:
def inductive_split(g):
    """Split the graph into training graph, validation graph, and test graph by training
    and validation masks.  Suitable for inductive models."""
    train_g = g.subgraph(g.ndata["train_mask"])
    val_g = g.subgraph(g.ndata["train_mask"] | g.ndata["val_mask"])
    test_g = g
    return train_g, val_g, test_g

In [16]:
fan_out=[10, 25]
sampler = dgl.dataloading.MultiLayerNeighborSampler([int(_) for _ in fan_out])

In [17]:
sampler

In [25]:
g.ndata.keys()

dict_keys(['train_mask', 'val_mask', 'test_mask', 'features', 'labels'])

In [18]:
help(g.formats)

Help on method formats in module dgl.heterograph:

formats(formats=None) method of dgl.heterograph.DGLHeteroGraph instance
    Get a cloned graph with the specified sparse format(s) or query
    for the usage status of sparse formats
    
    The API copies both the graph structure and the features.
    
    If the input graph has multiple edge types, they will have the same
    sparse format.
    
    Parameters
    ----------
    formats : str or list of str or None
    
        * If formats is None, return the usage status of sparse formats
        * Otherwise, it can be ``'coo'``/``'csr'``/``'csc'`` or a sublist of
          them, specifying the sparse formats to use.
    
    Returns
    -------
    dict or DGLGraph
    
        * If formats is None, the result will be a dict recording the usage
          status of sparse formats.
        * Otherwise, a DGLGraph will be returned, which is a clone of the
          original graph with the specified sparse format(s) ``formats``.
    

## 2. nagetive sampler